In [15]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn

In [16]:
df = pd.read_csv('local_file.csv')

In [17]:
df['type'].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [18]:
df['isFraud'].isnull().sum()

0

In [19]:
import pandas as pd
df['type'] = df['type'].map(df['type'].value_counts(normalize=False))

print(df.head())


   step     type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1  2151495   9839.64  C1231006815       170136.0       160296.36   
1     1  2151495   1864.28  C1666544295        21249.0        19384.72   
2     1   532909    181.00  C1305486145          181.0            0.00   
3     1  2237500    181.00   C840083671          181.0            0.00   
4     1  2151495  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [20]:
df['nameOrig'].value_counts()

C1530544995    3
C545315117     3
C724452879     3
C1784010646    3
C1677795071    3
              ..
C1567523029    1
C644777639     1
C1256645416    1
C1231536757    1
C1971151096    1
Name: nameOrig, Length: 6353307, dtype: int64

In [21]:
df['nameOrig'] = df['nameOrig'].astype(str).str[1:].astype(int)

print(df.head())

   step     type    amount    nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1  2151495   9839.64  1231006815       170136.0       160296.36   
1     1  2151495   1864.28  1666544295        21249.0        19384.72   
2     1   532909    181.00  1305486145          181.0            0.00   
3     1  2237500    181.00   840083671          181.0            0.00   
4     1  2151495  11668.14  2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [22]:
df['nameDest'] = df['nameDest'].astype(str).str[1:].astype(int)

print(df.head())



   step     type    amount    nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1  2151495   9839.64  1231006815       170136.0       160296.36   
1     1  2151495   1864.28  1666544295        21249.0        19384.72   
2     1   532909    181.00  1305486145          181.0            0.00   
3     1  2237500    181.00   840083671          181.0            0.00   
4     1  2151495  11668.14  2048537720        41554.0        29885.86   

     nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  1979787155             0.0             0.0        0               0  
1  2044282225             0.0             0.0        0               0  
2   553264065             0.0             0.0        1               0  
3    38997010         21182.0             0.0        1               0  
4  1230701703             0.0             0.0        0               0  


In [23]:
import pandas as pd
columns_to_drop = ['oldbalanceDest', 'oldbalanceOrg','isFlaggedFraud']

df = df.drop(columns=columns_to_drop)

In [24]:
X= df.drop(columns=['isFraud'])
y=df['isFraud']

In [25]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state = 42)


In [29]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [31]:
train_index = X_train.index
train_columns = X_train.columns
test_index = X_test.index
test_columns = X_test.columns

# Convert the scaled arrays back to DataFrames
X_train_df = pd.DataFrame(X_train_scaled, index=train_index, columns=train_columns)
X_test_df = pd.DataFrame(X_test_scaled, index=test_index, columns=test_columns)

# Convert the target series to DataFrames
y_train_df = pd.DataFrame(y_train, index=X_train.index)
y_test_df = pd.DataFrame(y_test, index=X_test.index)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
import os
import boto3

# Define your bucket and prefix paths
bucket = 'udacity-nano-degree-project'
prefix = 'processed_data'

# Save the DataFrames as CSV files
X_train_df.to_csv('X_train.csv', header=True, index=False)
y_train_df.to_csv('y_train.csv', header=True, index=False)
X_test_df.to_csv('X_test.csv', header=True, index=False)
y_test_df.to_csv('y_test.csv', header=True, index=False)

# Get the S3 client
s3 = boto3.client('s3')

# Upload the files to S3
s3.upload_file('X_train.csv', bucket, f'{prefix}/X_train.csv')
s3.upload_file('y_train.csv', bucket, f'{prefix}/y_train.csv')
s3.upload_file('X_test.csv', bucket, f'{prefix}/X_test.csv')
s3.upload_file('y_test.csv', bucket, f'{prefix}/y_test.csv')

# Now your data is in S3 and you can set up the s3_input objects
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/X_train.csv', content_type='csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/X_test.csv', content_type='csv')


In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Set your SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # Make sure this IAM role has the necessary permissions

# Path to your training script
script_path = 'training_script.py'

# Create SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point=script_path,
    role=role,
    instance_type='ml.m5.xlarge',  # Example instance type, choose one based on your needs
    framework_version='0.23-1',  # Match this to your scikit-learn version
    py_version='py3',  # Choose Python version
    sagemaker_session=sagemaker_session
)

# Fit estimator
sklearn_estimator.fit({'train': s3_input_train, 'test': s3_input_test})

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import randint as sp_randint

# Define the parameter space that will be searched over
param_dist = {
    'n_estimators': sp_randint(50, 200),  # Instead of a list, we define a range
    'max_depth': [None, 10, 20, 30],  # Fixed list is still fine here
    'min_samples_split': sp_randint(2, 10),  # Define a range for random integers
    'min_samples_leaf': sp_randint(1, 4)  # Define another range for random integers
}

# Instantiate a RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Set up the cross-validation scheme
stratified_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)

# Set up the RandomizedSearchCV object
n_iter_search = 20  # Number of parameter settings that are sampled
random_search = RandomizedSearchCV(estimator=rf_classifier, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search, 
                                   cv=stratified_kfold, 
                                   scoring='f1', 
                                   n_jobs=-1, 
                                   random_state=42)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict the labels for the test set
y_pred = best_model.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=42)
stratified_kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=stratified_kfold, scoring='f1', n_jobs=-1)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
 
y_pred = best_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train= smote.fit_resample(X_train, y_train)


svm_classifier = SVC(class_weight=class_weights)
param_grid = {
    'C': [0.1, 1, 10], 
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
}
stratified_kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=stratified_kfold, scoring='f1', n_jobs=-1, )
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)